In [11]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cvxpy as cp

#Import CSV and do categorical preprocessing
cols = list(pd.read_csv('garments_worker_productivity.csv', nrows=1))
df = pd.read_csv('garments_worker_productivity.csv', usecols = [i for i in cols if i != 'date'])
df.head(5)

,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382


In [12]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   quarter                1197 non-null   object 
 1   department             1197 non-null   object 
 2   day                    1197 non-null   object 
 3   team                   1197 non-null   int64  
 4   targeted_productivity  1197 non-null   float64
 5   smv                    1197 non-null   float64
 6   wip                    691 non-null    float64
 7   over_time              1197 non-null   int64  
 8   incentive              1197 non-null   int64  
 9   idle_time              1197 non-null   float64
 10  idle_men               1197 non-null   int64  
 11  no_of_style_change     1197 non-null   int64  
 12  no_of_workers          1197 non-null   float64
 13  actual_productivity    1197 non-null   float64
dtypes: float64(6), int64(5), object(3)
memory usage: 131.0+ 

In [29]:
#Categorical features: day, quarter, department, team
days = list(set(df.day))
quarters = list(set(df.quarter))
depts = list(set(df.department))
teams = list(set(df.team))

print(days, quarters, depts, teams, sep="\n")

['Monday', 'Saturday', 'Thursday', 'Sunday', 'Tuesday', 'Wednesday']
['Quarter3', 'Quarter2', 'Quarter4', 'Quarter5', 'Quarter1']
['finishing', 'finishing ', 'sweing']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [89]:
df.groupby(df.columns, 1, sort=False).first()
column = pd.Series(df['department'])
column.str.replace('sweing ', 'sewing')

df['department'] = df['department'].str.replace('sweing', 'sewing')
df['department'] = df['department'].str.replace('finishing ', 'finishing')

#Add target column 'satisfied'
zeros = pd.Series(np.zeros(df.shape[0],))
df['satisfied'] = zeros

def eval_satisfied(actual, targeted):
    if(actual  >= targeted):
        return 1
    else:
        return 0
    
for index, row in df.iterrows():
    actual = row['actual_productivity']
    target = row['targeted_productivity']
    #print(act_perf, tar_perf)
    #print(row['satisfied'])
    #print(eval_satisfied(actual, target))
    df.at[index, 'satisfied'] = eval_satisfied(actual, target)
df.head(25)

,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity,satisfied
0,Quarter1,sewing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725,1.0
1,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500,1.0
2,Quarter1,sewing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570,1.0
3,Quarter1,sewing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570,1.0
4,Quarter1,sewing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382,1.0
5,Quarter1,sewing,Thursday,7,0.80,25.90,984.0,6720,38,0.0,0,0,56.0,0.800125,1.0
6,Quarter1,finishing,Thursday,2,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.755167,1.0
7,Quarter1,sewing,Thursday,3,0.75,28.08,795.0,6900,45,0.0,0,0,57.5,0.753683,1.0
8,Quarter1,sewing,Thursday,2,0.75,19.87,733.0,6000,34,0.0,0,0,55.0,0.753098,1.0
9,Quarter1,sewing,Thursday,1,0.75,28.08,681.0,6900,45,0.0,0,0,57.5,0.750428,1.0
